Get Fact Data

In [0]:
%sql

drop table silver.workflowfact;

create table silver.workflowfact as
SELECT
fpol.PolicyNumber
,whtasks.WorkflowHistoryTaskId
,whtasks.WorkflowHistoryId
,f.FileId
,doc.DocumentId
,fpol.DepartmentId as fpolDepartmentID
,cu.DepartmentID as cuDepartmentID
,uu.DepartmentID as uuDepartmentID
,wsteps.ResponsibleDepartmentId
,fpol.CompanyID
,fpol.EffectiveDate
,fpol.ExpirationDate
,whtasks.Initiated
,wsteps.WorkflowID
,wsteps.WorkflowStepID
,whtasks.DueDate
,whtasks.DueDateDate
,f.AgencyContactId
,f.RegionId
,whtasks.CompletedDate
,whtasks.Completed
,f.AgencyId
,whtasks.AssignedToUserId
,whtasks.CompletedByUserId
,wsteps.ExcludeFromMaster
,whtasks.LastModified
,f.deleted as fileDeleted
,fpol.Deleted as fpolDeleted
,doc.deleted as docDeleted,
DATEDIFF(day, whtasks.Initiated,whtasks.Completed) AS DaysTakenToComplete


FROM bronze.tb_dm_workflowhistorytasks_spark whtasks
INNER JOIN bronze.tb_dm_WorkflowHistory_spark as wh on wh.WorkflowHistoryId = whtasks.WorkflowHistoryId
INNER JOIN bronze.tb_dm_WorkflowSteps_spark as wsteps on wsteps.WorkflowStepID = whtasks.WorkflowStepId
LEFT JOIN bronze.tb_shared_users_spark as uu on uu.UserId = whtasks.AssignedToUserId
LEFT JOIN bronze.tb_shared_users_spark cu on cu.UserId = whtasks.CompletedByUserId
INNER JOIN bronze.tb_dm_Document_spark as doc on doc.DocumentId = wh.DocumentId --Can we not get to document id through another table we are already pulling so we can drop wh?
LEFT JOIN bronze.tb_dm_File_spark as f on f.FileId = doc.FileId
LEFT JOIN bronze.tb_dm_FilePolicies_spark as fpol ON fpol.FilePolicyId = doc.FilePolicyId
INNER JOIN bronze.tb_dm_Workflows_spark as w ON w.WorkflowID = wsteps.WorkflowId
LEFT JOIN bronze.tb_am_Agencies_spark as a ON a.AgencyId = f.AgencyId
LEFT JOIN bronze.tb_shared_Regions_spark as r ON r.RegionId = f.RegionId;